In [2]:
import logging
import ray
from ray import tune
import ray.tune.progress_reporter as reporter
from typing import Optional, Type
from ray.rllib.models.torch.torch_modelv2 import TorchModelV2
from ray.rllib.models.torch.misc import normc_initializer, SlimFC, SlimConv2d
from ray.rllib.utils.annotations import override
from ray.rllib.utils.framework import try_import_torch
from ray.rllib.models.catalog import ModelCatalog
from ray.rllib.utils.typing import ModelConfigDict
import ray.rllib.agents.ddpg as ddpg
import numpy as np
from ray.rllib.models.torch.misc import SlimFC

In [2]:
torch, nn = try_import_torch()
ray.init()

2021-08-13 00:25:58,611	INFO services.py:1245 -- View the Ray dashboard at http://127.0.0.1:8265


{'node_ip_address': '192.168.1.72',
 'raylet_ip_address': '192.168.1.72',
 'redis_address': '192.168.1.72:6379',
 'object_store_address': '/tmp/ray/session_2021-08-13_00-25-57_145541_38969/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2021-08-13_00-25-57_145541_38969/sockets/raylet',
 'webui_url': '127.0.0.1:8265',
 'session_dir': '/tmp/ray/session_2021-08-13_00-25-57_145541_38969',
 'metrics_export_port': 57852,
 'node_id': 'a2ceb2142f9f6297b9936dc16bf8f19557f999d3cacaf4b64eb4713a'}

In [ ]:
############################
#### Environment
############################

class Corridor(gym.Env):
    """Example of a custom env in which you have to walk down a corridor.
    You can configure the length of the corridor via the env config."""

    def __init__(self, config: EnvContext):
        self.end_pos = config["corridor_length"]
        self.cur_pos = 0
        self.n = 100
        self.action_space = np.linspace(-1, 1, self.n)
        self.observation_space = np.linspace(-1, 1, self.n)
        # Set the seed. This is only used for the final (reach goal) reward.
#         self.seed(config.worker_index * config.num_workers)

    def reset(self):
        self.cur_pos = 0
        return [self.cur_pos]

    def step(self, action):
        assert action in [0, 1], action
        if action == 0 and self.cur_pos > 0:
            self.cur_pos -= 1
        elif action == 1:
            self.cur_pos += 1
        done = self.cur_pos >= self.end_pos
        # Produce a random reward when we reach the goal.
        return [self.cur_pos], \
            random.random() * 2 if done else -0.1, done, {}

    def seed(self, seed=None):
        random.seed(seed)

In [14]:
#######################################################
###### MODEL: LSTM + linear
#######################################################

class Net(TorchModelV2, nn.Module):
    def __init__(self, obs_space, action_space, num_outputs, model_config,
                 name):
        
        TorchModelV2.__init__(self, obs_space, action_space, num_outputs, model_config, name)
        nn.Module.__init__(self)
        
        custom_configs = model_config.get("custom_model_config")

        self.size_in = 32; ## 16+16
        self.lstm_state_size = 64
        self.grab_location = 1
        self.lstm = nn.LSTM(self.size_in, self.lstm_state_size, batch_first=True)
        self.num_outputs = self.grab_location + 7 ## one grab location + seven (position + translation)

        # Postprocess LSTM output with another hidden layer and compute values.
        self.linear = SlimFC(self.lstm_state_size, self.num_outputs)
        
            

    def forward(self, inputs, state, seq_lens):
        ## input will be (size 32, 16 for desired trajectory + another 16 for final trajectory config)
        ## state will be (grasp locations + position + orientation) current
        
        batch_size = x.shape[0] 
        x = self.flatten(x)
        x = torch.reshape(x, (batch_size, 100, 16+16)) 
        
        # Forward through LSTM.
        output, [h, c] = self.lstm(x, state)
        
        # Forward LSTM output
        output = self.linear(h[-1,...])
        return nn.Tanh()(output)  ## squeeze the output from -1 to 1
        
        

ModelCatalog.register_custom_model("my_model", Net)
model = {"custom_model":"my_model"}


In [15]:
#######################################################
##### DDPG config
#######################################################

config = ddpg.DEFAULT_CONFIG.copy()
train_batch_size = 200  ## batch size will be tuned later

config_ddpg = {
    
    "num_gpus": 4,
    "lr": 1e-5, ## starting low
    "monitor": True,
    "model":model,
    "train_batch_size":train_batch_size,
    "lambda":0.95,
    "clip_actions":True,
    "normalize_actions":True,
    "batch_mode": "complete_episodes", ## or "truncate_episodes"
    "no_done_at_end":True,
    "shuffle_buffer_size": 200, ## need to be tuned
    "framework": "torch",
    "corridor_length": 1,
#     "env": environment, ## need to be import from custom
    
    
}
config.update(config_ddpg)


In [16]:
def stopper(trial_num, result):
    ### determine when the training will stop
    #(when mean reward larger than 50 or episode larger than 1500 will stop train)
    return result["episode_reward_mean"] > 50 and result["episode_len_mean"] > 1500

In [17]:
### training

tune.run("DDPG", 
        config = config,
#         local_dir = SAVE_RESULTS_PATH, ## need to be set path when we trained on beast
        checkpoint_freq=25,
        verbose=2,
        checkpoint_at_end=True,
        stop=stopper,
#         progress_reporter=reporter ## need to be set later
        )

2021-08-13 00:51:05,334	WARNING tune.py:507 -- SIGINT received (e.g. via Ctrl+C), ending Ray Tune run. This will try to checkpoint the experiment state one last time. Press CTRL+C one more time (or send SIGINT/SIGKILL/SIGTERM) to skip. 


Trial name,status,loc
DDPG_None_6fc1e_00000,PENDING,


2021-08-13 00:51:05,545	ERROR tune.py:546 -- Trials did not complete: [DDPG_None_6fc1e_00000]
2021-08-13 00:51:05,547	INFO tune.py:550 -- Total run time: 5.52 seconds (5.25 seconds for the tuning loop).
2021-08-13 00:51:05,548	WARNING tune.py:554 -- Experiment has been interrupted, but the most recent state was saved. You can continue running this experiment by passing `resume=True` to `tune.run()`
